## 1. Library Import

In [7]:


# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import requests
import time
#import missingno as msno
import warnings;warnings.filterwarnings('ignore')


# dataframe 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import lightgbm as lgb


## 2. Data Load

In [8]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = '../data/dt_coord.csv'
test_path  = '../data/dt_test_coord.csv'
dt = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)
dt_original = dt.copy()
dt_test_original = dt_test.copy()


In [9]:
# Train과 Test data를 살펴보겠습니다.
display(dt.head(1))
display(dt_test.head(1))      # 부동산 실거래가(=Target) column이 제외된 모습입니다.
# display(dt_bus.head(1))
# display(dt_subway.head(1))   

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,,-,-,아파트,025776611,025776673,NaN,분양,자치관리,계단식,개별난방,8.0,270.0,우성건설,모름,1987-11-21 00:00:00.0,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,NaN,2022-11-09 20:10:43.0,2023-09-23 17:21:41.0,NaN,직영,단일계약,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000


,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,202307,26,5,1987,언주로 3,NaN,,직거래,-,아파트,025776611,025776673,NaN,분양,자치관리,계단식,개별난방,8.0,270.0,우성건설,모름,1987-11-21 00:00:00.0,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,NaN,2022-11-09 20:10:43.0,2023-09-23 17:21:41.0,NaN,직영,단일계약,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0


#### 2.2 데이터 탐색
- 데이터 탐색에는 데이터의 크기, 변수, 변수 유형을 확인합니다.
- 변수 유형의 경우 데이터를 직접 구별하여 범주형, 숫자형, 시간형으로 분류합니다. 

In [10]:
# Train data와 Test data shape은 아래와 같습니다.
print('Train data shape : ', dt.shape, 'Test data shape : ', dt_test.shape)
print("Train Test ratio:", dt_test.shape[0]/dt.shape[0])

Train data shape :  (1109340, 52) Test data shape :  (9272, 51)
Train Test ratio: 0.008358122847819425


In [11]:
# Train data와 Test data 변수는 아래와 같습니다.
print(len(dt.columns),"가지의 변수들")
print(dt.columns)

print(len(dt_test.columns),"가지의 변수들")
print(dt_test.columns)


52 가지의 변수들
Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target'],
      dtype='object')
51 가지의 변수들
Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적'

In [12]:
print("변수별 종류 수")
for variable in dt.columns:
    print(variable,":", len(dt[variable].unique()))


변수별 종류 수
시군구 : 338
번지 : 6530
본번 : 1523
부번 : 329
아파트명 : 6486
전용면적(㎡) : 14545
계약년월 : 198
계약일 : 31
층 : 73
건축년도 : 60
도로명 : 9140
해제사유발생일 : 1026
등기신청일자 : 182
거래유형 : 3
중개사소재지 : 643
k-단지분류(아파트,주상복합등등) : 6
k-전화번호 : 1542
k-팩스번호 : 1665
단지소개기존clob : 95
k-세대타입(분양형태) : 4
k-관리방식 : 4
k-복도유형 : 6
k-난방방식 : 5
k-전체동수 : 42
k-전체세대수 : 522
k-건설사(시공사) : 345
k-시행사 : 556
k-사용검사일-사용승인일 : 674
k-연면적 : 735
k-주거전용면적 : 740
k-관리비부과면적 : 736
k-전용면적별세대현황(60㎡이하) : 349
k-전용면적별세대현황(60㎡~85㎡이하) : 388
k-85㎡~135㎡이하 : 245
k-135㎡초과 : 2
k-홈페이지 : 222
k-등록일자 : 127
k-수정일자 : 744
고용보험관리번호 : 527
경비비관리형태 : 5
세대전기계약방법 : 3
청소비관리형태 : 5
건축면적 : 456
주차대수 : 527
기타/의무/임대/임의=1/2/3/4 : 5
단지승인일 : 736
사용허가여부 : 2
관리비 업로드 : 3
좌표X : 9114
좌표Y : 9114
단지신청일 : 260
target : 14361


## 3. Data Preprocessing

#### 3.1. 결측치 탐색 및 보간
- 본 데이터는 업스테이지에서 제공한 데이터로 변수에 대한 설명이 극히 제한적입니다. 변수에 대한 설명이 있었다면 결측치가 있는 변수를 제거하는 방식 외에 다른 보간법을 판단하기가 매우 어렵스럽습니다.
- 그리하여, 결측치가 많은 변수(열) 삭제를 결정하였습니다. 

- 의미를 알 수 없는 값을 확인하여 처리합니다. 

In [13]:
# 이레 처럼 아무 의미도 갖지 않는 칼럼은 결측치와 같은 역할을 하므로, np.nan으로 채워 결측치로 인식되도록 합니다.
dt['거래유형'].value_counts().head()

거래유형
-       1077014
중개거래      29228
직거래        3098
Name: count, dtype: int64

In [14]:
dt['등기신청일자'].value_counts().head()

등기신청일자
            1094431
20230630        585
20230428        394
20230831        391
20230530        367
Name: count, dtype: int64

In [15]:
dt['중개사소재지'].value_counts().head()

중개사소재지
-         1080142
서울 강남구       2000
서울 송파구       1995
서울 노원구       1966
서울 강동구       1562
Name: count, dtype: int64

- 위 처럼 아무 의미도 갖지 않는 칼럼은 결측치와 같은 역할을 하므로, np.nan으로 채워 결측치로 인식되도록 합니다.


In [16]:
# NaN으로 변환
dt['거래유형'] = dt['거래유형'].replace('-', np.nan)
dt['등기신청일자'] = dt['등기신청일자'].replace(' ', np.nan)
dt['중개사소재지'] = dt['중개사소재지'].replace('-', np.nan)

In [17]:
no_na_cols = dt.columns[dt.isna().sum() == 0].tolist()
has_na_cols = dt.columns[dt.isna().sum() > 0].tolist()

print("결측치가 없는 변수 개수 :", len(no_na_cols))
print("결측치가 없는 변수 목록 :", no_na_cols)
print("결측치가 있는 변수 개수 :", len(has_na_cols))

unselected = dt.columns[dt.isnull().sum() > 0]

dt_select = dt.drop(columns=unselected)
dt_test_select = dt_test.drop(columns=unselected)

print("결측치 제거 후 train data shape:", dt_select.shape)
print("결측치 제거 후 test data shape:", dt_test_select.shape)


dt = dt_select
dt_test = dt_test_select

결측치가 없는 변수 개수 : 10
결측치가 없는 변수 목록 : ['시군구', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '도로명', '좌표X', '좌표Y', 'target']
결측치가 있는 변수 개수 : 42
결측치 제거 후 train data shape: (1109340, 10)
결측치 제거 후 test data shape: (9272, 9)


- test 데이터의 결측치도 확인합니다.
- 확인 결과, 결측치가 없으므로 결측치 보간을 마무리 합니다. 

In [18]:
no_na_cols = dt_test.columns[dt_test.isna().sum() == 0].tolist()
has_na_cols = dt_test.columns[dt_test.isna().sum() > 0].tolist()

print("결측치가 없는 변수 개수 :", len(no_na_cols))
print("결측치가 없는 변수 목록 :", no_na_cols)
print("결측치가 있는 변수 개수 :", len(has_na_cols))


결측치가 없는 변수 개수 : 7
결측치가 없는 변수 목록 : ['시군구', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '도로명']
결측치가 있는 변수 개수 : 2


### 3.2. 이상치 처리
- 이상치를 처리하기 위해 변수의 범위를 확인합니다.

In [19]:
num_cols = [
    '전용면적(㎡)',
    '층',
    '건축년도'
]

cat_cols = [
    '시군구',
    '본번',
    '부번',
    '도로명'
]

date_cols = [
    '등기신청일자',
]

In [20]:
dt[num_cols].describe().round(2)


,전용면적(㎡),층,건축년도
count,1109340.00,1109340.00,1109340.00
mean,77.24,8.91,1998.88
std,29.38,5.99,9.25
min,10.02,-4.00,1961.00
25%,59.67,4.00,1993.00
50%,82.19,8.00,2000.00
75%,84.96,12.00,2005.00
max,424.32,69.00,2023.00


In [21]:
dt_test[num_cols].describe().round(2)


,전용면적(㎡),층,건축년도
count,9272.00,9272.00,9272.00
mean,75.41,10.02,2003.03
std,29.34,6.65,11.18
min,12.00,-3.00,1961.00
25%,59.70,5.00,1996.00
50%,76.57,9.00,2003.00
75%,84.96,14.00,2012.00
max,301.47,65.00,2023.00


- Numerical 변수의 이상치 비율을 확인합니다.
- 전용면적의 경우 이상치 비율이 7.98% 이지만, 최대/최소, std 등 크게 범위를 넘어가지 않아 삭제를 하지 않기로 합니다.

## 4. Feature Engineering
- 이제 파생변수를 만들어보도록 하겠습니다.
- 파생 변수는 도메인 지식에 기반해 제작하는 것이 중요합니다.
- 주택의 가격을 예측하는 문제이기 때문에 약간의 부동산 지식을 기반으로 파생 변수를 만들어 보도록 하겠습니다.

In [23]:
# 시군구, 년월 등 분할할 수 있는 변수들은 세부사항 고려를 용이하게 하기 위해 모두 분할해 주겠습니다.
dt['구'] = dt['시군구'].map(lambda x : x.split()[1])
dt['동'] = dt['시군구'].map(lambda x : x.split()[2])
del dt['시군구']

dt['계약년'] = dt['계약년월'].astype('str').map(lambda x : x[:4])
dt['계약월'] = dt['계약년월'].astype('str').map(lambda x : x[4:])
# del dt['계약년월']

dt_test['구'] = dt_test['시군구'].map(lambda x : x.split()[1])
dt_test['동'] = dt_test['시군구'].map(lambda x : x.split()[2])
del dt_test['시군구']

dt_test['계약년'] = dt_test['계약년월'].astype('str').map(lambda x : x[:4])
dt_test['계약월'] = dt_test['계약년월'].astype('str').map(lambda x : x[4:])
# dt_test['계약년월']

- 서울의 집값은 강남, 강북 여부에 따라 차이가 많이 난다는 사실은 많이 알려진 사실입니다.
- 따라서 강남/강북의 여부에 따라 파생변수를 생성해주도록 하겠습니다.

In [24]:
all = list(dt['구'].unique())
gangnam = ['강서구', '영등포구', '동작구', '서초구', '강남구', '송파구', '강동구']
gangbuk = [x for x in all if x not in gangnam]

assert len(all) == len(gangnam) + len(gangbuk)       # 알맞게 분리되었는지 체크합니다.

In [25]:
# # 강남의 여부를 체크합니다.
is_gangnam = []
for x in dt['구'].tolist() :
  if x in gangnam :
    is_gangnam.append(1)
  else :
    is_gangnam.append(0)

# 파생변수를 하나 만릅니다.
dt['강남여부'] = is_gangnam


# 강남의 여부를 체크합니다.
is_gangnam = []
for x in dt_test['구'].tolist() :
  if x in gangnam :
    is_gangnam.append(1)
  else :
    is_gangnam.append(0)

# 파생변수를 하나 만릅니다.
dt_test['강남여부'] = is_gangnam

- 또한 신축인지, 구축인지의 여부도 실거래가에 큰 영향을 줄 수 있습니다.
- 따라서 건축년도에 따라 파생변수를 제작해주도록 하겠습니다.

In [26]:
# 따라서 2009년 이후에 지어졌으면 비교적 신축이라고 판단하고, 신축 여부 변수를 제작해보도록 하겠습니다.
dt['신축여부(2009년)'] = dt['건축년도'].apply(lambda x: 1 if x >= 2009 else 0)
dt_test['신축여부(2009년)'] = dt_test['건축년도'].apply(lambda x: 1 if x >= 2009 else 0)

In [27]:
dt.head(1)       # 최종 데이터셋은 아래와 같습니다.

,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,좌표X,좌표Y,target,구,동,계약년,계약월,강남여부,신축여부(2009년)
0,79.97,201712,8,3,1987,언주로 3,127.05721,37.476763,124000,강남구,개포동,2017,12,1,0


In [28]:
dt_test.head(1)

,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,좌표X,좌표Y,구,동,계약년,계약월,강남여부,신축여부(2009년)
0,79.97,202307,26,5,1987,언주로 3,127.05721,37.476763,강남구,개포동,2023,07,1,0


In [29]:
dt.shape

(1109340, 15)

In [30]:
dt_test.shape

(9272, 14)

## 5. Model Training

In [31]:
dt_train = dt
dt_test['target'] = 0

In [32]:

col_order = ["계약년월", "계약년", "계약월", "계약일", "건축년도", "신축여부(2009년)", "구", "동", "도로명", "좌표X", "좌표Y", "강남여부", "전용면적(㎡)", "층", "target" ]

dt_train = dt_train[col_order]
dt_test = dt_test[col_order]


In [33]:
dt_train.head(1)

,계약년월,계약년,계약월,계약일,건축년도,신축여부(2009년),구,동,도로명,좌표X,좌표Y,강남여부,전용면적(㎡),층,target
0,201712,2017,12,8,1987,0,강남구,개포동,언주로 3,127.05721,37.476763,1,79.97,3,124000


In [34]:
dt_test.head(1)

,계약년월,계약년,계약월,계약일,건축년도,신축여부(2009년),구,동,도로명,좌표X,좌표Y,강남여부,전용면적(㎡),층,target
0,202307,2023,07,26,1987,0,강남구,개포동,언주로 3,127.05721,37.476763,1,79.97,5,0


In [35]:
# Train data와 Test data shape은 아래와 같습니다.
print('Train data shape : ', dt.shape, 'Test data shape : ', dt_test.shape)


Train data shape :  (1109340, 15) Test data shape :  (9272, 15)


In [36]:
train_roads = set(dt_train["도로명"].dropna().unique())
test_roads = set(dt_test["도로명"].dropna().unique())
len(train_roads & test_roads)
only_train = train_roads - test_roads
overlap_ratio = len(train_roads & test_roads) / len(test_roads)
overlap_ratio
summary = {
    "train_unique": len(train_roads),
    "test_unique": len(test_roads),
    "common": len(train_roads & test_roads),
    "only_train": len(train_roads - test_roads),
    "only_test": len(test_roads - train_roads),
    "test_overlap_ratio": len(train_roads & test_roads) / len(test_roads),
}

summary

{'train_unique': 9140,
 'test_unique': 2642,
 'common': 2627,
 'only_train': 6513,
 'only_test': 15,
 'test_overlap_ratio': 0.9943224829674489}

In [37]:
cnt = dt_test["도로명"].isin(test_roads - train_roads).sum()
print(cnt)


76


In [38]:


# 시간형 변수 명시
time_columns = ["계약년월", "계약년", "계약월", "계약일"]

continuous_columns_v2 = []
categorical_columns_v2 = []
time_columns_v2 = []

for column in dt_train.columns:
    if column in time_columns:
        time_columns_v2.append(column)
    elif pd.api.types.is_numeric_dtype(dt_train[column]):
        if (column == "target"):
            continue
        continuous_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("연속형 변수:", continuous_columns_v2)
print("범주형 변수:", categorical_columns_v2)
print("시간형 변수:", time_columns_v2)


연속형 변수: ['건축년도', '신축여부(2009년)', '좌표X', '좌표Y', '강남여부', '전용면적(㎡)', '층']
범주형 변수: ['구', '동', '도로명']
시간형 변수: ['계약년월', '계약년', '계약월', '계약일']


### 5.2. Model Training
- 위 데이터를 이용해 모델을 train 해보겠습니다. 모델은 RandomForest를 이용하겠습니다.
- Train과 Valid dataset을 분할하는 과정에서는 `holdout` 방법을 사용하겠습니다. 이 방법의 경우  대략적인 성능을 빠르게 확인할 수 있다는 점에서 baseline에서 사용해보도록 하겠습니다.
  - 이 후 추가적인 eda를 통해서 평가세트와 경향을 맞추거나 kfold와 같은 분포에 대한 고려를 추가할 수 있습니다.

In [39]:
assert dt_train.shape[1] == dt_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.

In [40]:
# Target과 독립변수들을 분리해줍니다.
y_train = dt_train['target']
X_train = dt_train.drop(['target'], axis=1)

# Hold out split을 사용해 학습 데이터와 검증 데이터를 8:2 비율로 나누겠습니다.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)

In [41]:
# 시계열 기준 분할
train_mask = dt_train['계약년월'] <= 202206
valid_mask = ~train_mask

feature_cols = dt_train.drop(columns=['target']).columns.tolist()
X_train = dt_train.loc[train_mask, feature_cols]
y_train = dt_train.loc[train_mask, 'target']

X_val = dt_train.loc[valid_mask, feature_cols]
y_val = dt_train.loc[valid_mask, 'target']

X_test = dt_test[feature_cols]


print("시계열 Validation 분할 결과")
print("="*60)
print(f"Train: {X_train.shape[0]:,}건     [기간] {dt_train.loc[train_mask, '계약년월'].min()} ~ {dt_train.loc[train_mask, '계약년월'].max()}")
print(f"Valid: {X_val.shape[0]:,}건        [기간] {dt_train.loc[valid_mask, '계약년월'].min()} ~ {dt_train.loc[valid_mask, '계약년월'].max()}")
print(f"Test:  {X_test.shape[0]:,}건         [기간] {dt_test['계약년월'].min()} ~ {dt_test['계약년월'].max()}")




시계열 Validation 분할 결과
Train: 1,087,682건     [기간] 200701 ~ 202206
Valid: 21,658건        [기간] 202207 ~ 202306
Test:  9,272건         [기간] 202307 ~ 202309


In [42]:
# "계약년월", "계약일"을 드랍합니다.
X_train = X_train.drop(["계약년월", "계약일"], axis=1)
X_val   = X_val.drop(["계약년월", "계약일"], axis=1)
X_test  = X_test.drop(["계약년월", "계약일"], axis=1)
time_columns_v2 = [c for c in time_columns_v2 if c not in ["계약년월", "계약일"]]


print("연속형 변수:", continuous_columns_v2)
print("범주형 변수:", categorical_columns_v2)
print("시간형 변수:", time_columns_v2)

연속형 변수: ['건축년도', '신축여부(2009년)', '좌표X', '좌표Y', '강남여부', '전용면적(㎡)', '층']
범주형 변수: ['구', '동', '도로명']
시간형 변수: ['계약년', '계약월']


In [43]:
for col in ["계약년", "계약월"]:
    X_train[col] = X_train[col].astype(int)
    X_val[col] = X_val[col].astype(int)
    X_test[col] = X_test[col].astype(int)


# 범주형 변수: ['구', '동', '도로명']
for col in categorical_columns_v2:
    X_train[col] = X_train[col].astype("category")
    X_val[col] = X_val[col].astype("category")
    X_test[col] = X_test[col].astype("category")

# 문자열 컬럼 제거한 버전 (RF / Ridge / XGB 용)
X_train_num = X_train.drop(columns=categorical_columns_v2)
X_val_num   = X_val.drop(columns=categorical_columns_v2)
X_test_num  = X_test.drop(columns=categorical_columns_v2)


In [47]:
# ================================
# IMPORTS
# ================================
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor


# ================================
# MODEL DEFINITIONS
# ================================
models = {
    "rf": RandomForestRegressor(
        n_estimators=300,
        max_depth=20,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1
    ),

    #"ridge": Ridge(alpha=1.0),

    "xgb": xgb.XGBRegressor(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1
    ),

    "lgb": lgb.LGBMRegressor(
        n_estimators=500,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ),

    "cat": CatBoostRegressor(
        iterations=500,
        learning_rate=0.05,
        depth=6,
        loss_function="MAE",
        random_seed=42,
        verbose=False
    )
}


# ================================
# TRAIN / VALID / TEST
# ================================
val_scores = {}
val_preds = {}
test_preds = {}

for name, model in models.items():
    print(f"\n===== TRAIN {name.upper()} =====")

    if name in ["rf", "xgb"]:
        Xtr, Xv, Xte = X_train_num, X_val_num, X_test_num
    else:
        Xtr, Xv, Xte = X_train, X_val, X_test

    if name == "cat":
        model.fit(
            Xtr,
            y_train,
            cat_features=categorical_columns_v2
        )
    else:
        model.fit(Xtr, y_train)

    val_pred = model.predict(Xv)
    test_pred = model.predict(Xte)

    mae = mean_absolute_error(y_val, val_pred)

    val_scores[name] = mae
    val_preds[name] = val_pred
    test_preds[name] = test_pred

    print(f"{name.upper()} VAL MAE: {mae:.4f}")



# ================================
# VALIDATION RESULT
# ================================
val_result = pd.DataFrame({
    "model": list(val_scores.keys()),
    "val_mae": list(val_scores.values())
}).sort_values("val_mae")

print("\n===== VALIDATION SUMMARY =====")
print(val_result)


# ================================
# ENSEMBLE (NaN-SAFE)
# ================================
best_models = val_result["model"].values[:3]

valid_ensemble_models = []
for m in best_models:
    if not np.isnan(val_preds[m]).any():
        valid_ensemble_models.append(m)
    else:
        print(f"⚠️ {m} has NaN predictions → excluded")

print("\nFINAL ENSEMBLE MODELS:", valid_ensemble_models)

val_ensemble_pred = np.mean(
    [val_preds[m] for m in valid_ensemble_models],
    axis=0
)

ensemble_val_mae = mean_absolute_error(y_val, val_ensemble_pred)
print(f"\n ENSEMBLE VAL MAE: {ensemble_val_mae:.4f}")


# ================================
# TEST ENSEMBLE & SUBMISSION
# ================================
test_ensemble_pred = np.mean(
    [test_preds[m] for m in valid_ensemble_models],
    axis=0
)

submission = pd.DataFrame({
    "target": test_ensemble_pred
})

submission.to_csv("submission.csv", index=False)
print("\nsubmission.csv saved")



===== TRAIN RF =====
RF VAL MAE: 18557.6544

===== TRAIN XGB =====
XGB VAL MAE: 20011.2637

===== TRAIN LGB =====
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011295 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7363
[LightGBM] [Info] Number of data points in the train set: 1087682, number of used features: 12
[LightGBM] [Info] Start training from score 57100.713850
LGB VAL MAE: 19810.7508

===== TRAIN CAT =====
CAT VAL MAE: 20640.0114

===== VALIDATION SUMMARY =====
  model       val_mae
0    rf  18557.654405
2   lgb  19810.750816
1   xgb  20011.263672
3   cat  20640.011445

FINAL ENSEMBLE MODELS: ['rf'